In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from os import path, getcwd
from sqlalchemy import create_engine

In [2]:
DBS_DIR = "databases"
def get_database_path(extension="csv"):
    db_path = f"{DBS_DIR}/database.{extension}"
    return path.normpath(path.join(path.split(getcwd())[0], db_path)).replace("\\", "/")

In [3]:
DB_PATH = get_database_path()
df = pd.read_csv(DB_PATH)

In [5]:
new_columns = {
    'ID':"id",
    'Пол':"gender", 
    'Дата рождения':"birthdate", 
    'Субъект РФ':"subject", 
    'Населенный пункт':"locality", 
    'Группа инвалидности':"disability_group", 
    'Причина инвалидности':"disability_cause",
    'Этиология потери зрения':"sight_loss_etiology", 
    'Возраст потери зрения':"sight_loss_age",
    'Состояние зрения':"sight_condition",
    'Причины потери зрения':"sight_loss_cause",
    'Этиология потери слуха':"hearing_loss_etiology", 
    'Возраст потери слуха':"hearing_loss_age",
    'Состояние слуха':"hearing_condition",
    'Причины потери слуха':"hearing_loss_cause",
    'Знает синдром':"syndrome_awareness", 
    'Название синдрома':"syndrome",
    'Наличие других заболеваний':"other_diseases",
    'Диагноз':"diagnosis", 
    'Понимаемые языки':"recognizable_languages", 
    'Восприятие информации':"information_perception",
    'Передача информации':"communication_methods", 
    'Источники информации':"information_sources", 
    'Потребность в помощи':"assistance_necessity",
    'Передвижение по улице':"street_movement", 
    'Необходимые ТСР':"rehabilitation_technologies", 
    'Использование переводчика':"translator_assistance",
    'Тип образования':"education", 
    'Ученая степень':"academic_degree", 
    'Специальность':"specialization",
    'Работаю в данный момент':"working", 
    'Желание работать':"working_willingness", 
    'Условия проживания':"living_conditions"
}
df = df.rename(columns=new_columns)

In [6]:
df["syndrome_awareness"] = df["syndrome_awareness"].str.replace("Нет, синдром не установлен или неизвестен", "Нет")
df["syndrome_awareness"] = df["syndrome_awareness"].str.replace("Да, синдром известен", "Да")

In [7]:
df["birthdate"] = df["birthdate"].str.replace('.', "/", regex=False)
df["birthdate"] = df["birthdate"].str.replace('0000', "2000", regex=False)
df["birthdate"] = pd.to_datetime(df["birthdate"], dayfirst=True)

In [9]:
df.replace(r'^[Дд]а$', True, regex=True, inplace=True)
df.replace(r'^[Нн]ет$', False, regex=True, inplace=True)

In [10]:
boolean_variables = ["syndrome_awareness", "other_diseases", "working", "working_willingness"]
df[boolean_variables] = df[boolean_variables].astype("boolean")

In [25]:
persons_columns = df.columns[1:5]
persons = df.loc[:, persons_columns]

In [27]:
disability_information_columns = df.columns[5:19]
disability_information = df.loc[:, disability_information_columns]

In [28]:
additional_information_columns = df.columns[19:]
additional_information = df.loc[:, additional_information_columns]

In [30]:
engine = create_engine(f"sqlite:///{get_database_path('db')}", echo=False)
with engine.begin() as connection:
    persons.to_sql('persons', con=connection, if_exists='replace', index_label='id')
    disability_information.to_sql('disability_information', con=connection, if_exists='replace', index_label='person_id')
    additional_information.to_sql('additional_information', con=connection, if_exists='replace', index_label='person_id')